In [1]:
import pandas as pd
import re

data = pd.read_csv('data.tsv', sep = '\t', keep_default_na = False, low_memory=False)

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

def delSW(text):
  stop_words = stopwords.words('english')
  stop_words.extend([',','.',';',':','(',')',"'",'s'])
  return [word for word in text if word not in stop_words]

def lem(text):
  wnl = WordNetLemmatizer()
  return [wnl.lemmatize(w) for w in text]

def stem(text):
  array = []
  for word in text:
    stemmer = SnowballStemmer('english')
    word = stemmer.stem(word)
    array.append(word)
  return array

nltk_data = data

# nltk_data['tokened'] = nltk_data.abstract.str.lower().apply(word_tokenize).apply(delSW)
# nltk_data['lemmed'] = nltk_data.tokened.apply(lem)
# nltk_data['stemmed'] = nltk_data.tokened.apply(stem)

nltk_data.to_csv('nltk_data_prepped.tsv', sep="\t")

In [7]:
from collections import Counter
Counter(''.join(str(v).replace('"','')
  .replace("'",'')
  .replace('[','')
  .replace(']','')
  .replace(',','') for v in nltk_data.lemmed).split()).most_common(10)

[('firm', 59073),
 ('study', 35965),
 ('s', 35157),
 ('model', 34665),
 ('effect', 33078),
 ('result', 29513),
 ('research', 25971),
 ('market', 25653),
 ('find', 24762),
 ('performance', 24605)]

In [3]:
import re
gensim_data = data
gensim_data['abst'] = gensim_data.abstract.map(lambda x: re.sub(r'[^\w\s]', '', x))
gensim_data['abst'] = gensim_data.abst.map(lambda x: x.lower())
gensim_data.abst.head()
gensim_data.to_csv('gensim_data_prepped.tsv', sep="\t")

In [100]:
import gensim
import spacy
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words_gensim = stopwords.words('english')
stop_words_gensim.extend(['from', 'subject', 're', 'edu', 'use', 'research', 'study'])

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(text):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_gensim] for doc in text]

nlp = spacy.load("en_core_web_sm")
abstracts_isolated = gensim_data.abst.apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))

In [103]:

abstracts_isolated_edited = abstracts_isolated.values.tolist()
abstracts_words_isolated = list(sent_to_words(abstracts_isolated_edited))
abstracts_words_isolated = remove_stopwords(abstracts_isolated_edited)

In [101]:
abstracts_isolated.to_csv('lemmed.tsv', sep = '\t')

In [104]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(abstracts_words_isolated)
corpus = [id2word.doc2bow(text) for text in abstracts_words_isolated]
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word, num_topics = 10)

In [106]:
from pyLDAvis import gensim_models as gensimvis
from pyLDAvis import enable_notebook, save_html
enable_notebook()

model = gensimvis.prepare(lda_model, corpus, id2word)
save_html(model, 'viz.html')
model

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.040004  0.017048       1        1  17.098327
7      0.010293 -0.024419       2        1  15.707222
0      0.009192 -0.015748       3        1  10.310231
4     -0.034482  0.018799       4        1   9.739040
9     -0.003772  0.004687       5        1   9.666138
2     -0.009871 -0.023624       6        1   8.955595
8     -0.001862  0.023323       7        1   7.862246
6     -0.009499 -0.014225       8        1   7.698196
1      0.009438  0.009999       9        1   7.451019
3     -0.009439  0.004160      10        1   5.511985, topic_info=            Term          Freq         Total Category  logprob  loglift
186         firm  62776.000000  62776.000000  Default  30.0000  30.0000
88      consumer  16882.000000  16882.000000  Default  29.0000  29.0000
194        price  16957.000000  16957.000000  Default  28.0000  28.0000
573      service   9099.000000   9099.000000  Default  27.0000  27.0000
262  information  23167.000000  23167.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
138      examine   1088.411554  19962.186975  Topic10  -5.7302  -0.0109
256         find   1271.857709  31730.964438  Topic10  -5.5744  -0.3186
964     analysis    922.899641  13805.478815  Topic10  -5.8951   0.1929
185    financial    868.485020  11402.813760  Topic10  -5.9559   0.3234
329         time    870.077398  14277.174760  Topic10  -5.9541   0.1004

[987 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
3105       1  0.474941   abnormal
3105       2  0.055090   abnormal
3105       3  0.049247   abnormal
3105       4  0.052586   abnormal
3105       5  0.062602   abnormal
...      ...       ...        ...
13177      8  0.024838  wrongdoer
13177      9  0.049675  wrongdoer
3324       1  0.873520         wu
3324       8  0.116469         wu
1948       7  0.885239    zeroday

[4223 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 1, 5, 10, 3, 9, 7, 2, 4])

In [8]:
with open(model, 'wb') as f:
        pickle.dump(model, f)

NameError: name 'model' is not defined